# Developing a model architecture for life expectancy classification

## Packages to use

In [ ]:
## Imports go here
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.utils import plot_model
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import nibabel as nib
import scikeras

## Working with data

In [ ]:
## empty for now for data import or whatever
#need two X's: X_nii for neuroimaging and X_age for age
#for first optional model X_nii and X_age can be united into X_train

## Model architecture - CNN

In [ ]:
input_shape = (240, 240, 155, 1) ##probably we need to change it because of age

In [ ]:
#we need 1 kernel as we have 1 label for segmentation, kernel size = 3x3x3?
# https://stackoverflow.com/questions/42556919/adding-a-variable-into-keras-tensorflow-cnn-dense-layer
# https://stackoverflow.com/questions/43196636/how-to-concatenate-two-layers-in-keras
#initialize model

def initialize_model(dropout = 0.5, dense_1 = 50, \
    learning_rate = 0.01, kernel_size=(3,3,3), pool_size = (2,2,2)):
    model = Sequential()
    
    #Add convo layers to the model
    model.add(Conv3D(32, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Conv3D(64, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Conv3D(128, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    
    #Add a flatten layer
    model.add(Flatten())
    
    #Add dense levels
    model.add(Dense(dense_1, activation='relu'))
    model.add(Dropout(dropout))
    
    #maybe ADD AGE here for the second model
    # Merge the output of the convNet with your added features by concatenation
    model_age_input = Sequential()
    model_age_input.add(Dense(1, input_shape=(1,), activation='relu'))
    
    # concatenate two layers
    model_with_age = Concatenate([model, model_age_input])
    
    #Add layer with activation
    model_with_age.add(Dense(16, activation='relu'))
    model_with_age.add(Dense(3, activation='softmax'))
    
    #Model compilation
    optim=Adam(learning_rate=learning_rate)
    model_with_age.compile(loss = 'categorical_crossentropy',
                  optimizer = optim,
                  metrics = ['accuracy'])
    return model_with_age


In [ ]:
#instantiate a model
model_seg = initialize_model()
model_seg.summary()

In [ ]:
#better to write it down as a function

In [ ]:
#baseline model score
es = EarlyStopping(patience=3, restore_best_weights = True)
history = model_seg.fit(X_train, y_train,
                        epochs = 30,
                        batch_size = 16,
                        callbacks = [es],
                        validation_split = 0.2,
                        verbose = 1)

In [ ]:
#plot the learning curve
def plot_loss(history):
    fig, (ax1, ax2) = plt.subplots(1,2, figsize=(13,4))
    ax1.plot(history.history['loss'])
    ax1.plot(history.history['val_loss'])
    ax1.set_title('Model loss')
    ax1.set_ylabel('Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylim(ymin=0, ymax=200)
    ax1.legend(['Train', 'Validation'], loc='best')
    ax1.grid(axis="x",linewidth=0.5)
    ax1.grid(axis="y",linewidth=0.5)    
    
    ax2.plot(history.history['accuracy'])
    ax2.plot(history.history['val_accuracy'])
    ax2.set_title('Accuracy')
    ax2.set_ylabel('Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylim(ymin=0, ymax=20)
    ax2.legend(['Train', 'Validation'], loc='best')
    ax2.grid(axis="x",linewidth=0.5)
    ax2.grid(axis="y",linewidth=0.5)    

    plt.show()    

In [ ]:
keras_estimator = KerasClassifier(build_fn = initialize_model, verbose = 1)

In [ ]:
estimator = Pipeline([('kc', keras_estimator)])

In [ ]:
estimator.get_params()


In [ ]:
#hyperparameters tuning
# Define the hyperparameters
param_grid = {
    'kc__dense_1': [20, 30, 50, 100],
    'kc__kernel_size': [(2,2,2),(3,3,3), (5,5,5), (7,7,7)],
    'kc__pool_size': [(2,2,2),(3,3,3)],
    'kc__batch_size':[8, 16, 32],
    'kc__dropout': [0.5, 0.4, 0.3, 0.2, 0.1, 0],
    'kc__learning_rate': [0.001, 0.01, 0.1]
}


In [ ]:
kfold_splits = 5
grid = GridSearchCV(estimator=estimator,  
                    n_jobs=-1, 
                    verbose=1,
                    return_train_score=True,
                    cv=kfold_splits,  #StratifiedKFold(n_splits=kfold_splits, shuffle=True)
                    param_grid=param_grid,)

In [ ]:
grid_result = grid.fit(X, y) #callbacks=[tbCallBack]

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#prediction with the final model 
model_seg.evaluate(X_test, y_test)

In [ ]:
#Second model architecture  option in case the first approach doesn't work
def build_model_nii(dropout = 0.5, dense_1 = 50, \
    learning_rate = 0.01, kernel_size=(3,3,3), pool_size = (2,2,2)):
    model = Sequential()
    #Add convo layers to the model
    model.add(Conv3D(32, kernel_size=kernel_size, activation='relu', input_shape=input_shape))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Conv3D(64, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    model.add(Conv3D(128, kernel_size=kernel_size, activation='relu'))
    model.add(MaxPooling3D(pool_size=pool_size))
    
    #Add a flatten layer
    model.add(Flatten())
    
    #Add dense levels
    model.add(Dense(dense_1, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(3, activation='softmax'))
    
    optim=Adam(learning_rate=0.01)
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = optim,
                  metrics = ['accuracy'])
    return model

model_nii = build_model_nii()
model_nii.summary()

In [ ]:
es = EarlyStopping(patience=3, restore_best_weights = True)

model_nii = build_model_nii()
model_nii.fit(X_nii, y, 
          validation_split=0.3,
          epochs=30, 
          batch_size=16,
          callbacks=[es]
          )

In [ ]:
def build_model_age():
    input_age = Input(shape=(X_age.shape[1],))

    x = Dense(64, activation="relu")(input_age)
    x = Dense(32, activation="relu")(x)
    output_age = Dense(1, activation="relu")(x)

    model_age = Model(inputs=input_age, outputs=output_age)
    
    return model_age

model_age = build_model_age()
model_age.summary()

In [ ]:

optim=Adam(learning_rate=0.01)
model_age.compile(loss = 'categorical_crossentropy',
                  optimizer = optim,
                  metrics = ['accuracy'])
model_age.fit(X_age, y, 
          validation_split=0.3,
          epochs=30, 
          batch_size=16,
          callbacks=[es]
          )

In [ ]:
# Define Inputs and Outputs of nii model as with age Model

#model_nii = build_model_nii() # comment-out to keep pre-trained weights not to start from scratch
input_nii = model_nii.input
output_nii = model_nii.output

#model_age = build_model_age() # comment-out to keep pre-trained weights not to start from scratch
input_age = model_age.input
output_age = model_age.output

In [ ]:
# Let's combine the two streams of data and add two dense layers on top!
inputs = [input_nii, input_age]

combined = layers.concatenate([output_nii, output_age])

x = Dense(16, activation="relu")(combined)

outputs = Dense(3, activation="softmax")(x)

model_combined = Model(inputs=inputs, outputs=outputs)

In [ ]:
model_combined.summary()

In [ ]:
plot_model(model_combined, "multi_input_model.png", show_shapes=True)

In [ ]:
model_combined.compile(loss = 'categorical_crossentropy',
                  optimizer = optim,
                  metrics = ['accuracy'])
es = EarlyStopping(patience=3, restore_best_weights = True)

model_combined.fit(x=[X_nii, X_age], 
                   y=y,
                   validation_split=0.3,
                   epochs=50,
                   batch_size=16,
                   callbacks=[es])

In [ ]:
#pleasedefine X_test and y_test before
model_combined.evaluate(X_test, y_test)